## Etherscan Data Merging

In [138]:
# Import packages 
import json
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import requests
import time
from dotenv import dotenv_values
config = dotenv_values(".env")
APIKEY = config["APIKEY"]

### Utils

In [21]:
# Now that we have a list of all transactions 
# we will start looking at "from" and "to" tokens for each transactions(normal and internal). 

# Normal transaction by address 
def getNormalTranTxlist(address,APIKEY):
    resp = requests.get("http://api.etherscan.io"+
                    "/api?"+
                    "module=account&"+
                    "action=txlist&"+
                    "address="+ address+ "&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

# Internal Transaction by address 
def getInternalTranTxlist(address,APIKEY):
    resp = requests.get("http://api.etherscan.io"+
                    "/api?"+
                    "module=account&"+
                    "action=txlistinternal"+
                    "&address="+ address+ "&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

# All transactions by address 
def getTranTokennfttx(address,APIKEY):
    resp = requests.get(
                    "http://api.etherscan.io" +
                    "/api?"+ 
                    "module=account&"+
                    "action=tokennfttx&"+
                    "&address="+ address+ "&"+
                    "page=1&"+
                    "offset=10000&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

In [49]:
# API Calls
'''
Transactions From addresses 
 Function can be one of three:
     1. internal -> getInternalTranTxlist
     2. norm     -> getNormalTranTxlist
     3. nfttx    -> getTranTokennfttx
'''
def getTxList(addresses,APIKEY,func):
    start = time.time()
    APIKEY = APIKEY
    succesful_dict              = []
    unsuccessful_dict = []
    print("Working on ", func)

    for i in tqdm(addresses):
        nest_start = time.time()
        if i != "0x0000000000000000000000000000000000000000":
            # Choose a function
            if   func == "internal": 
                resp = getInternalTranTxlist(i,APIKEY)
            elif func == "norm":
                resp = getNormalTranTxlist(i,APIKEY)
            elif func == "nfttx": 
                resp = getTranTokennfttx(i,APIKEY)

            # check if status pases
            if resp['status'] != '1':
#                 print("Address: ",i,"\nMessage:" ,resp['message'])
                unsuccessful_dict = unsuccessful_dict + [{"address": i,
                                                                 "message": resp['message']}]
            else:
                # Transactions
                succesful_dict = succesful_dict+[{"Address": i, "data": resp}]
            
        # Set filler timer to avoid reaching API calls maxlimit
        nest_end = time.time()
        fill = 0.200 - (nest_end-nest_start)
        if fill > 0:
            time.sleep(fill+0.05)

    end = time.time()
    print("Total Time Spent:", end - start)

    return succesful_dict, unsuccessful_dict



In [73]:
# Check Length
def findNOTOK(data):
    matched = []
    for i in data:
        if i['message'] == 'NOTOK':
            matched += [i]
    return matched

def checkLength(nfttx_dict,unsuccessful_nfttx_dict,internal_dict,unsuccessful_internal_dict,norm_dict,unsuccessful_norm_dict):
# There are to different types of missing data. Either there simply isn't any transaction, or timeout.
    print("length of nfttx_dict:                          ", len(nfttx_dict))
    print("length of unsuccessful_nfttx_dict/LengthNOTOK: ", len(unsuccessful_nfttx_dict),
                                                            "/",
                                                            len(findNOTOK(unsuccessful_nfttx_dict)))

    print("length of internal_dict:                       ", len(internal_dict))
    print("length of unsuccessful_internal_dict/NOTOK:    ", len(unsuccessful_internal_dict),
                                                            "/",
                                                            len(findNOTOK(unsuccessful_internal_dict)))

    print("length of norm_dict:                           ", len(norm_dict))
    print("length of unsuccessful_norm_dict/NOTOK:        ", len(unsuccessful_norm_dict),
                                                             "/",
                                                             len(findNOTOK(unsuccessful_norm_dict)))

In [117]:
def convert_to_df(final_dict,func):
    # Loop through entire from_internal_dict_old to pull values
    # from address, and api call status 
    # func can be "nfttx," "txlist," "txlistinternal"
    print("Working on:",func)
    result = []
    # converted_df = pd.DataFrame()
    unsuccesful_df = []
    j = 0
    for i in tqdm(final_dict):
        if i['data']['status'] == '1':
            out = pd.DataFrame(i['data']['result'])
            out["address"] = i['Address']
            if func   == "nfttx":
                out["apicall"] = "tokennfttx"

            elif func == "txlist":
                out["apicall"] = "txlist"

            elif func == "txlistinternal":
                out["apicall"] = "txlistinternal"
            
            # changed to concat instead of append
            # converted_df = converted_df.append(out)
            result.append(out)

        else:
            print(j)
            unsuccesful_df = unsuccesful_df +[i]
            j+=1
    return pd.concat(result), unsuccesful_df

##### Concat vs. Append

In [103]:
import numpy as np
df = pd.DataFrame(np.random.randn(100000,20))
df['B'] = 'foo'
df['C'] = pd.Timestamp('20130101')

In [104]:
def f1():
    result = df
    for i in range(9):
         result = result.append(df)
    return result

In [105]:
def f2():
    result = []
    for i in range(10):
        result.append(df)
    return pd.concat(result)


In [106]:
f1().equals(f2())

True

In [107]:
%timeit f1()

681 ms ± 8.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [108]:
%timeit f2()

101 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Load Data

In [2]:
# Opening JSON file
f = open('../DATA/bigdata/nfttx_dict.json')
# returns JSON object as
# a dictionary
nfttx_dict = json.load(f)
f.close()

# Opening JSON file
f = open('../DATA/bigdata/internal_dict.json')
# returns JSON object as
# a dictionary
internal_dict = json.load(f)
f.close()

# Opening JSON file
f = open('../DATA/bigdata/norm_dict.json')
# returns JSON object as
# a dictionary
norm_dict = json.load(f)
f.close()



In [3]:
# Opening JSON file
f = open('../DATA/bigdata/unsuccessful_nfttx_dict.json')
# returns JSON object as
# a dictionary
unsuccessful_nfttx_dict = json.load(f)
f.close()

# Opening JSON file
f = open('../DATA/bigdata/unsuccessful_internal_dict.json')
# returns JSON object as
# a dictionary
unsuccessful_internal_dict = json.load(f)
f.close()

# Opening JSON file
f = open('../DATA/bigdata/unsuccessful_norm_dict.json')
# returns JSON object as
# a dictionary
unsuccessful_norm_dict = json.load(f)
f.close()

### Explore Missing Data 

In [74]:
checkLength(nfttx_dict,
            unsuccessful_nfttx_dict,
            internal_dict,
            unsuccessful_internal_dict,
            norm_dict,
            unsuccessful_norm_dict)

length of nfttx_dict:                           7198
length of unsuccessful_nfttx_dict/LengthNOTOK:  75 / 75
length of internal_dict:                        5398
length of unsuccessful_internal_dict/NOTOK:     1875 / 91
length of norm_dict:                            6993
length of unsuccessful_norm_dict/NOTOK:         280 / 16


In [80]:
# Rerun Addresses that recieved NOTOK
nfttx_dict_leftover, unsc_nfttx_dict_leftover = getTxList([d['address'] for d in findNOTOK(unsuccessful_nfttx_dict)],
                                      APIKEY,
                                      "internal")
internal_dict_leftover,unsc_internal_dict_leftover = getTxList([d['address'] for d in findNOTOK(unsuccessful_internal_dict)],
                                      APIKEY,
                                      "nfttx")
norm_dict_leftover,unsc_norm_dict_leftover = getTxList([d['address'] for d in findNOTOK(unsuccessful_norm_dict)],
                                      APIKEY,
                                      "norm")
final_nfttx_dict    = nfttx_dict + nfttx_dict_leftover
final_internal_dict = internal_dict + internal_dict_leftover
final_norm_dict     = norm_dict + norm_dict_leftover

Working on  internal


100%|███████████████████████████████████████████| 75/75 [00:22<00:00,  3.40it/s]


Total Time Spent: 22.04499626159668
Working on  nfttx


100%|███████████████████████████████████████████| 91/91 [00:26<00:00,  3.44it/s]


Total Time Spent: 26.427623987197876
Working on  norm


100%|███████████████████████████████████████████| 16/16 [00:04<00:00,  3.89it/s]

Total Time Spent: 4.11700701713562


### Convert to DF

In [110]:
final_nfttx_df, unsuccesful_final_nfttx_df         = convert_to_df(final_nfttx_dict,"nfttx")
final_internal_df, unsuccessful_final_internal_df  = convert_to_df(final_internal_dict,"txlistinternal")
final_norm_df, unsuccessful_final_norm_df          = convert_to_df(final_norm_dict,"txlist")

Working on: nfttx


100%|██████████████████████████████████████| 7255/7255 [00:15<00:00, 473.07it/s]


Working on: txlistinternal


100%|██████████████████████████████████████| 5489/5489 [00:07<00:00, 687.91it/s]


Working on: txlist


100%|██████████████████████████████████████| 7009/7009 [00:18<00:00, 377.55it/s]


In [118]:
final_nfttx_df.to_csv("./DATA/bigdata/final_nfttx_df.csv")
final_internal_df.to_csv("./DATA/bigdata/final_internal_df.csv")
final_norm_df.to_csv("./DATA/bigdata/final_norm_df.csv")

### Left Join to create One Dataset
This will exclude oneway hanging branch nodes

In [120]:
nfttx_internal_df=pd.merge(final_norm_df,final_nfttx_df, on="hash", how="left")

In [121]:
final_three_combined_df=pd.merge(nfttx_internal_df,final_internal_df, on="hash", how="left")

In [127]:
final_three_combined_df.to_csv("./DATA/bigdata/final_three_combined_df.csv")